In [4]:
!ipc/ipc-static

ipc-static [-Bqsv] [-b buffersize] [-i pipe|local|tcp] [-p tcp_port]
	[-P l1d|l1i|l2|mem|tlb|axi] [-t totalsize] mode

Modes (pick one - default 1thread):
    1thread                IPC within a single thread
    2thread                IPC between two threads in one process
    2proc                  IPC between two threads in two different processes

Optional flags:
    -B                     Run in bare mode: no preparatory activities
    -i pipe|local|tcp      Select pipe, local sockets, or TCP (default: pipe)
    -p tcp_port            Set TCP port number (default: 10141)
    -P l1d|l1i|l2|mem|tlb|axi  Enable hardware performance counters
    -q                     Just run the benchmark, don't print stuff out
    -s                     Set send/receive socket-buffer sizes to buffersize
    -v                     Provide a verbose benchmark description
    -b buffersize          Specify a buffer size (default: 131072)
    -t totalsize           Specify total I/O size (default: 1677

In [5]:
!ipc/ipc-static -i pipe 2thread
!ipc/ipc-static -i local 2thread

274948.53 KBytes/sec
135210.06 KBytes/sec


In [6]:
!ipc/ipc-static -i local -s -b 1024 2thread

46141.98 KBytes/sec


In [7]:
!ipc/ipc-static -v -i pipe 2thread

Benchmark configuration:
  buffersize: 131072
  totalsize: 16777216
  blockcount: 128
  mode: 2thread
  ipctype: pipe
  time: 0.059394292
275851.42 KBytes/sec


In [8]:
!ipc/ipc-static -q -B -i pipe 2thread

In [14]:

setup_kernel()

kern.ipc.maxsockbuf: 33554432 -> 33554432


## Things to do
- [ ] instrument clock_gettime for the IPC loop
- [ ] it may be sensible to inspect quantize() results for both the execution time distributions of the system calls, and the amount of data returned by each
- [ ] investigate scheduling events using the sched provider ( on-cpu and off-cpu events)
- [ ] instrument sleep and wakeup
- [ ] take care about pid and tid
- [ ] probe effect: one simple way to approach the problem is to analyse the results of performance benchmarking with and without DTrace scripts running
- [ ] discard the first result
- [ ] read the FreeBSD Benchmarking Advice wiki
### questions to answer
- How does increasing IPC buffer size uniformly change performance across IPC models – and why?
- Explore the impact of the probe effect on your causal investigation; how has DTrace changed the behavior of the benchmark?


In [52]:
from collections import defaultdict


def setup_kernel():
    !sysctl kern.ipc.maxsockbuf=33554432
# Callback invoked to print the trace record
# (that is, printf("%u", vtimestamp - self->start))

def benchmark(flags, trials, buff_sizes, dtrace_script,output_name="", quiet=False):
    values = []
    aggr_dict = defaultdict(int)

    def simple_out(value):
        values.append(value)
    
    def walk_func(a, b, c, d):
        pass
        #aggr_dict[c[0]] += d
    
    # print_header(["Starting io-static performance measurement",flags])

        # Create a seperate thread to run the DTrace instrumentation
    setup_kernel()
    dtrace_thread = DTraceConsumerThread(dtrace_script,
                                    chew_func=lambda v: None,
                                        chewrec_func=lambda v: None,
                                        out_func=simple_out,
                                        walk_func=walk_func,
                                        sleep=1)

    # Start the DTrace instrumentation
    dtrace_thread.start()

    # Display header to indicate that the benchmarking has started
    for size in buff_sizes:
        if not quiet:
            print("buffer size:", size, "collected so far: ",len(values))
        for i in range(trials):
            output = !ipc/ipc-static -B -q -b {str(size)} {flags} 2thread
            if len(output) > 0:
                print(output)
            
    # The benchmark has completed - stop the DTrace instrumentation
    dtrace_thread.stop()
    dtrace_thread.join()
    dtrace_thread.consumer.__del__()

    if not quiet:
        print("values collected:", len(values))

    if len(output_name) > 0:
        with open("lab2_{}_1202.data".format(output_name), 'w') as f:
            f.write(",".join([str(s) for s in BUFFER_SIZES]))
            f.write("\n")
            for value in values:
                f.write(value)
                f.write("\n")
                
    return values, aggr_dict

In [30]:
# IO performance varying the buffer size
ipc_trace_script = """

BEGIN {
}

syscall::clock_gettime:return
/execname == "ipc-static" && !self->in_benchmark/
{
    self->in_benchmark = 1;
    self->cstart = vtimestamp;
    printf("tid: %d, pid: %d, return clock_gettime", tid, pid);

}

syscall::clock_gettime:entry
/execname == "ipc-static" && self->in_benchmark/
{
    self->in_benchmark = 0;
    printf("tid: %d, pid: %d, time: %d", tid, pid, vtimestamp - self->cstart);
}

syscall::exit:entry
/execname == "ipc-static"/
{
    printf("----"); 
}

END
{
    exit(0);
}
"""
BUFFER_SIZES = [1024 * 2 ** exp for exp in range(0, 15)]


do_2_threads:

create second thread
sender: clock_gettime at the beginning
receiver: clock_gettime at the end

In [31]:

benchmark(
    flags="-i pipe",
    output_name="pipe",
    trials=1,
    buff_sizes=BUFFER_SIZES,
    dtrace_script=ipc_trace_script
)

kern.ipc.maxsockbuf: 33554432 -> 33554432
('buffer size:', 1024, 'collected so far: ', 0)
('buffer size:', 2048, 'collected so far: ', 0)
('buffer size:', 4096, 'collected so far: ', 0)
('buffer size:', 8192, 'collected so far: ', 0)
('buffer size:', 16384, 'collected so far: ', 0)
('buffer size:', 32768, 'collected so far: ', 15)
('buffer size:', 65536, 'collected so far: ', 15)
('buffer size:', 131072, 'collected so far: ', 15)
('buffer size:', 262144, 'collected so far: ', 15)
('buffer size:', 524288, 'collected so far: ', 15)
('buffer size:', 1048576, 'collected so far: ', 38)
('buffer size:', 2097152, 'collected so far: ', 38)
('buffer size:', 4194304, 'collected so far: ', 38)
('buffer size:', 8388608, 'collected so far: ', 38)
('buffer size:', 16777216, 'collected so far: ', 38)
('values collected:', 59)


(['tid: 100397, pid: 1169, return clock_gettime',
  'tid: 100128, pid: 1169, return clock_gettime',
  '----',
  'tid: 100398, pid: 1170, return clock_gettime',
  'tid: 100128, pid: 1170, time: 126964195',
  'tid: 100128, pid: 1170, return clock_gettime',
  '----',
  'tid: 100399, pid: 1171, return clock_gettime',
  'tid: 100128, pid: 1171, time: 100515164',
  'tid: 100128, pid: 1171, return clock_gettime',
  '----',
  'tid: 100400, pid: 1172, return clock_gettime',
  'tid: 100128, pid: 1172, time: 103860131',
  'tid: 100128, pid: 1172, return clock_gettime',
  '----',
  'tid: 100401, pid: 1173, return clock_gettime',
  'tid: 100128, pid: 1173, time: 99394304',
  'tid: 100128, pid: 1173, return clock_gettime',
  '----',
  'tid: 100402, pid: 1174, return clock_gettime',
  'tid: 100128, pid: 1174, time: 93994844',
  'tid: 100128, pid: 1174, return clock_gettime',
  '----',
  'tid: 100403, pid: 1175, return clock_gettime',
  'tid: 100128, pid: 1175, time: 106154534',
  'tid: 100128, pid: 1

In [33]:

benchmark(
    flags="-i local",
    output_name="local",
    trials=2,
    buff_sizes=BUFFER_SIZES,
    dtrace_script=ipc_trace_script
)

kern.ipc.maxsockbuf: 33554432 -> 33554432
('buffer size:', 1024, 'collected so far: ', 0)
('buffer size:', 2048, 'collected so far: ', 0)
('buffer size:', 4096, 'collected so far: ', 8)
('buffer size:', 8192, 'collected so far: ', 8)
('buffer size:', 16384, 'collected so far: ', 24)
('buffer size:', 32768, 'collected so far: ', 24)
('buffer size:', 65536, 'collected so far: ', 24)
('buffer size:', 131072, 'collected so far: ', 47)
('buffer size:', 262144, 'collected so far: ', 47)
('buffer size:', 524288, 'collected so far: ', 64)
('buffer size:', 1048576, 'collected so far: ', 64)
('buffer size:', 2097152, 'collected so far: ', 80)
('buffer size:', 4194304, 'collected so far: ', 80)
('buffer size:', 8388608, 'collected so far: ', 99)
('buffer size:', 16777216, 'collected so far: ', 99)
('values collected:', 119)


(['tid: 100429, pid: 1201, return clock_gettime',
  'tid: 100128, pid: 1201, return clock_gettime',
  '----',
  'tid: 100430, pid: 1202, return clock_gettime',
  'tid: 100128, pid: 1202, time: 224964072',
  'tid: 100128, pid: 1202, return clock_gettime',
  '----',
  'tid: 100431, pid: 1203, return clock_gettime',
  'tid: 100128, pid: 1203, time: 148954880',
  'tid: 100128, pid: 1203, return clock_gettime',
  '----',
  'tid: 100432, pid: 1204, return clock_gettime',
  'tid: 100128, pid: 1204, time: 150137394',
  'tid: 100128, pid: 1204, return clock_gettime',
  '----',
  'tid: 100433, pid: 1205, return clock_gettime',
  'tid: 100128, pid: 1205, time: 113198151',
  'tid: 100128, pid: 1205, return clock_gettime',
  '----',
  'tid: 100434, pid: 1206, return clock_gettime',
  'tid: 100128, pid: 1206, time: 112695213',
  'tid: 100128, pid: 1206, return clock_gettime',
  '----',
  'tid: 100435, pid: 1207, return clock_gettime',
  'tid: 100128, pid: 1207, time: 96021957',
  'tid: 100128, pid: 

In [45]:

benchmark(
    flags="-i local -s",
    output_name="socket_s",
    trials=2,
    buff_sizes=BUFFER_SIZES,
    dtrace_script=ipc_trace_script
)

kern.ipc.maxsockbuf: 33554432 -> 33554432
('buffer size:', 1024, 'collected so far: ', 0)
('buffer size:', 2048, 'collected so far: ', 0)
('buffer size:', 4096, 'collected so far: ', 11)
('buffer size:', 8192, 'collected so far: ', 11)
('buffer size:', 16384, 'collected so far: ', 11)
('buffer size:', 32768, 'collected so far: ', 31)
('buffer size:', 65536, 'collected so far: ', 31)
('buffer size:', 131072, 'collected so far: ', 52)
('buffer size:', 262144, 'collected so far: ', 52)
('buffer size:', 524288, 'collected so far: ', 68)
('buffer size:', 1048576, 'collected so far: ', 68)
('buffer size:', 2097152, 'collected so far: ', 80)
('buffer size:', 4194304, 'collected so far: ', 94)
('buffer size:', 8388608, 'collected so far: ', 94)
('buffer size:', 16777216, 'collected so far: ', 106)
('values collected:', 119)


(['tid: 100544, pid: 1368, return clock_gettime',
  'tid: 100128, pid: 1368, return clock_gettime',
  '----',
  'tid: 100545, pid: 1369, return clock_gettime',
  'tid: 100128, pid: 1369, time: 225246305',
  'tid: 100128, pid: 1369, return clock_gettime',
  '----',
  'tid: 100546, pid: 1370, return clock_gettime',
  'tid: 100128, pid: 1370, time: 151465698',
  'tid: 100128, pid: 1370, return clock_gettime',
  '----',
  'tid: 100547, pid: 1371, return clock_gettime',
  'tid: 100128, pid: 1371, time: 151343134',
  'tid: 100128, pid: 1371, return clock_gettime',
  '----',
  'tid: 100548, pid: 1372, return clock_gettime',
  'tid: 100128, pid: 1372, time: 113015241',
  'tid: 100128, pid: 1372, return clock_gettime',
  '----',
  'tid: 100549, pid: 1373, return clock_gettime',
  'tid: 100128, pid: 1373, time: 113355062',
  'tid: 100128, pid: 1373, return clock_gettime',
  '----',
  'tid: 100550, pid: 1374, return clock_gettime',
  'tid: 100128, pid: 1374, time: 96106267',
  'tid: 100128, pid: 

In [55]:
# IO performance varying the buffer size
syscall_distribution = """

BEGIN {
}

syscall::clock_gettime:return
/execname == "ipc-static" && !self->in_benchmark/
{
    self->in_benchmark = 1;
    self->cstart = vtimestamp;
   /* printf("tid: %d, pid: %d, return clock_gettime", tid, pid);*/

}

syscall::clock_gettime:entry
/execname == "ipc-static" && self->in_benchmark/
{
    self->in_benchmark = 0;
  /*  printf("tid: %d, pid: %d, time: %d", tid, pid, vtimestamp - self->cstart);*/
}

syscall::read:return 
/self->in_benchmark/
{
    @read = quantize(arg0)
}

syscall::write:return 
/self->in_benchmark/
{
    @write = quantize(arg0)
}

syscall::exit:entry
/execname == "ipc-static"/
{
    printf("Read aggregation:");
    printa(@read);
    printf("write aggregation:"); 
    printa(@write);
    
    clear(@read);
    clear(@write);

    printf("---------------------------"); 
}

END
{
    printf("hello world");
    exit(0);
}
"""
BUFFER_SIZES = [1024 * 2 ** exp for exp in range(0,15)]
out, dic = benchmark(
    flags="-i pipe",
    trials=1,
    buff_sizes=BUFFER_SIZES,
    dtrace_script=syscall_distribution
)
print("\n".join(out))

kern.ipc.maxsockbuf: 33554432 -> 33554432
('buffer size:', 1024, 'collected so far: ', 0)
('buffer size:', 2048, 'collected so far: ', 0)
('buffer size:', 4096, 'collected so far: ', 0)
('buffer size:', 8192, 'collected so far: ', 0)
('buffer size:', 16384, 'collected so far: ', 0)
('buffer size:', 32768, 'collected so far: ', 28)
('buffer size:', 65536, 'collected so far: ', 28)
('buffer size:', 131072, 'collected so far: ', 28)
('buffer size:', 262144, 'collected so far: ', 28)
('buffer size:', 524288, 'collected so far: ', 28)
('buffer size:', 1048576, 'collected so far: ', 63)
('buffer size:', 2097152, 'collected so far: ', 63)
('buffer size:', 4194304, 'collected so far: ', 63)
('buffer size:', 8388608, 'collected so far: ', 63)
('buffer size:', 16777216, 'collected so far: ', 91)
('values collected:', 105)
Read aggregation:
value  ------------- Distribution ------------- count    
              64 |                                         0        
             128 |             

In [46]:
out, dic = benchmark(
    flags="-i local",
    trials=1,
    buff_sizes=BUFFER_SIZES,
    dtrace_script=syscall_distribution
)
print("\n".join(out))

kern.ipc.maxsockbuf: 33554432 -> 33554432
('buffer size:', 1024, 'collected so far: ', 0)
('buffer size:', 2048, 'collected so far: ', 0)
('buffer size:', 4096, 'collected so far: ', 7)


Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored


('buffer size:', 8192, 'collected so far: ', 7)
('buffer size:', 16384, 'collected so far: ', 7)
('buffer size:', 32768, 'collected so far: ', 7)
('buffer size:', 65536, 'collected so far: ', 7)
('buffer size:', 131072, 'collected so far: ', 42)


Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored


('buffer size:', 262144, 'collected so far: ', 42)
('buffer size:', 524288, 'collected so far: ', 42)
('buffer size:', 1048576, 'collected so far: ', 63)


Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored


('buffer size:', 2097152, 'collected so far: ', 63)
('buffer size:', 4194304, 'collected so far: ', 63)
('buffer size:', 8388608, 'collected so far: ', 63)


Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored


('buffer size:', 16777216, 'collected so far: ', 91)
('values collected:', 105)
Read aggregation:
value  ------------- Distribution ------------- count    
              64 |                                         0        
             128 |                                         2        
             256 |                                         0        
             512 |                                         0        
            1024 |                                         0        
            2048 |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 7942     
            4096 |                                         0

write aggregation:
value  ------------- Distribution ------------- count    
             512 |                                         0        
            1024 |@@@@@@@@@@@@@@@@@@@@@@@@@@@              16384    
            2048 |@@@@@@@@@@@@@                            7942     
            4096 |                                         0

----
Read aggregation:

Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored


In [47]:
out, dic = benchmark(
    flags="-i local -s",
    trials=1,
    buff_sizes=BUFFER_SIZES,
    dtrace_script=syscall_distribution
)
print("\n".join(out))

kern.ipc.maxsockbuf: 33554432 -> 33554432
('buffer size:', 1024, 'collected so far: ', 0)
('buffer size:', 2048, 'collected so far: ', 0)
('buffer size:', 4096, 'collected so far: ', 0)


Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored


('buffer size:', 8192, 'collected so far: ', 14)
('buffer size:', 16384, 'collected so far: ', 14)
('buffer size:', 32768, 'collected so far: ', 14)
('buffer size:', 65536, 'collected so far: ', 14)
('buffer size:', 131072, 'collected so far: ', 42)


Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored


('buffer size:', 262144, 'collected so far: ', 42)
('buffer size:', 524288, 'collected so far: ', 42)
('buffer size:', 1048576, 'collected so far: ', 42)


Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored


('buffer size:', 2097152, 'collected so far: ', 70)
('buffer size:', 4194304, 'collected so far: ', 70)
('buffer size:', 8388608, 'collected so far: ', 70)


Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored


('buffer size:', 16777216, 'collected so far: ', 91)
('values collected:', 105)
Read aggregation:
value  ------------- Distribution ------------- count    
              64 |                                         0        
             128 |                                         2        
             256 |                                         0        
             512 |                                         0        
            1024 |                                         0        
            2048 |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 8192     
            4096 |                                         0

write aggregation:
value  ------------- Distribution ------------- count    
             512 |                                         0        
            1024 |@@@@@@@@@@@@@@@@@@@@@@@@@@@              16384    
            2048 |@@@@@@@@@@@@@                            8192     
            4096 |                                         0

----
Read aggregation:

Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored


In [48]:
out, dic = benchmark(
    flags="-i local -s",
    trials=1,
    buff_sizes=[BUFFER_SIZES[-1]],
    dtrace_script=syscall_distribution
)
print("\n".join(out))

kern.ipc.maxsockbuf: 33554432 -> 33554432
('buffer size:', 16777216, 'collected so far: ', 0)
('values collected:', 6)
Read aggregation:

write aggregation:
value  ------------- Distribution ------------- count    
         8388608 |                                         0        
        16777216 |@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 1        
        33554432 |                                         0

----


Exception IndexError: 'list index out of range' in 'dtrace.walk' ignored
